In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Url Context

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/url-context/intro_url_context.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Furl-context%2Fintro_url_context.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/url-context/intro_url_context.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/url-context/intro_url_context.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/url-context/intro_url_context.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/url-context/intro_url_context.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/url-context/intro_url_context.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/url-context/intro_url_context.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/url-context/intro_url_context.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Authors |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

The URL context tool in the Gemini API allows you to include URLs as context for your prompts. The model can then use the content from these URLs to generate more informed and relevant responses.

### Objectives

This tutorial provides a guide to using the URL context in the Gemini API.


## Getting Started

### Install the Google Gen AI SDK for Python


In [1]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Authenticate to Vertex AI on Google Cloud

You'll need to set up authentication by choosing **one** of the following methods:

1.  **Use a Google Cloud Project:** (Recommended for most users)
    - See instructions [Set up a project and development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)
2.  **Use a Vertex AI API Key (Express Mode):** For quick experimentation.
    - [Get an API Key](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview)
    - See tutorial [Getting started with Gemini using Vertex AI in Express Mode](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_express.ipynb).

This tutorial uses a Google Cloud Project for authentication.

In [3]:
import os

PROJECT_ID = "vipassna-proj"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global"

### Import libraries


In [5]:
from IPython.display import Markdown, display
from google import genai
from google.genai.types import GenerateContentConfig, GoogleSearch, Tool, UrlContext

### Connect to the generative AI service on Vertex AI

In [6]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Supported Models

Learn more about [supported Gemini models](https://cloud.google.com/vertex-ai/generative-ai/docs/models). This tutorial uses the Gemini 2.5 Flash model.

In [7]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## Use URL Context

The URL context tool is particularly useful for a variety of tasks, such as:

- Extracting data: You can pull key data points or talking points directly from articles.
- Comparing information: It allows for the comparison of information across multiple different links.
- Synthesizing data: The tool can be used to synthesize data from several different online sources.
- Answering questions: It's useful for answering questions based on the content found on one or more specific web pages.
- Content analysis: You can analyze content for a specific purpose, such as writing a job description or creating test questions from a webpage's text


#### **Example 1**: One URL only

The most direct way to use the feature is to provide a specific URL in your prompt that you want the model to analyze.

In [9]:
# Define the Url context tool
url_context_tool = Tool(url_context=UrlContext)

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Summarize this document: https://blog.google/technology/developers/introducing-gemini-cli-open-source-ai-agent/",
    config=GenerateContentConfig(
        tools=[url_context_tool],
    ),
)

display(Markdown(response.text))

Google has launched Gemini CLI, an open-source AI agent that integrates Gemini directly into the terminal for developers. This tool facilitates coding, problem-solving, and task management, offering free access to Gemini 2.5 Pro with a personal Google account, which includes a large context window and high usage limits. Developers can also use Google AI Studio or Vertex AI keys for more extensive access. Gemini CLI is designed to be extensible, supporting standards like the Model Context Protocol (MCP), and shares technology with Gemini Code Assist, providing AI assistance in both the terminal and VS Code.

**Inspect url context metadata**

When the model uses URLs to inform its answer, the response object will contain
`url_context_metadata`. This metadata lists the URLs the model retrieved and the status of that retrieval.

You can inspect this metadata to confirm which sources were used, as shown in the code example below. Inspecting this metadata is useful for debugging which sources were successfully retrieved or for verifying the model's information sources.


In [10]:
print(response.candidates[0].url_context_metadata)

url_metadata=[UrlMetadata(
  retrieved_url='https://blog.google/technology/developers/introducing-gemini-cli-open-source-ai-agent/',
  url_retrieval_status=<UrlRetrievalStatus.URL_RETRIEVAL_STATUS_SUCCESS: 'URL_RETRIEVAL_STATUS_SUCCESS'>
)]


### **Example 2**: Multiple URLs

The feature currently supports a maximum of 20 URLs per request.

In [ ]:
url_context_tool = Tool(url_context=UrlContext)

URL1 = "https://www.allrecipes.com/recipe/10813/best-chocolate-chip-cookies/"
URL2 = "https://www.allrecipes.com/recipe/17165/big-soft-ginger-cookies/"

prompt = f"Compare the ingredients and instructions from the recipe at {URL1} with the one at {URL2}. What are the key differences?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        tools=[url_context_tool],
    ),
)

display(Markdown(response.text))

print(response.candidates[0].url_context_metadata)

### **Example 3**: Grounding with Google Search

You can combine the URL context tool with Grounding with Google Search. This allows the model to first search for relevant information online and then use the URL context tool to get a deeper understanding of the content from the search results.

⚠️ This is a "Private/Allowlist" feature. You will need to require access before you can use it. Otherwise, you will get an 400 error "Multiple tools are supported only when they are all search tools."


In [12]:
# Define Url Context and Google Search tools
tools = [Tool(url_context=UrlContext), Tool(google_search=GoogleSearch())]

#### **Use Case**: Answering a question using search and a specific page

In this example, we ask the model to create a schedule based on a specific URL but also ask a follow-up question. The model can use the provided URL and perform a Google search to answer the entire prompt comprehensively.

In [ ]:
URL = "https://www.metmuseum.org/visit/plan-your-visit"

prompt = f"""Using the 'Exhibitions' and 'Must-See Artwork' sections of the URL {URL},
design a 2-hour art tour of the Met for someone primarily interested in European paintings.
Also, what are the museum's hours for next Saturday?
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        tools=tools,
    ),
)

display(Markdown(response.text))

print(response.candidates[0].url_context_metadata)

#### **Use Case**: Analyze an API docs guide

Developers constantly need to learn new APIs. This example demonstrates using the URL context to analyze API documentation and extract key information, making integration faster. This is a great way to synthesize data from several sources.

In [ ]:
URL = "https://jsonplaceholder.typicode.com/guide/"

prompt = f""""Analyze the API documentation guide at the provided URL {URL}.
Provide a summary of the available resource endpoints and list all the HTTP methods
that are supported for the '/posts' endpoint.
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        tools=tools,
    ),
)

display(Markdown(response.text))

print(response.candidates[0].url_context_metadata)

#### **Use Case**: Generate code based on a tutorial

This example shows how to use the URL context to read a programming tutorial and write functional code based on its concepts, a task that can dramatically speed up learning and development.

In [24]:
# URL = "https://realpython.com/python-requests/"

# prompt = f"""""Read the tutorial from the provided URL {URL}. Write a Python script
# that performs a GET request to 'https://api.github.com/users/google' and prints the
# user's name and number of public repositories. Include comments explaining each step
# and basic error handling for the request.
# """

URL = "https://www.cushmanwakefield.com/"

prompt = f"""""Extract all relevant URLs from the base website {URL} that list the company’s locations, such as branches, offices, stores, or facilities.
For example,return URLs for pages containing locations or addresses or offices or facilities or sites.
given output as array list of URLs separated by commas. like ['URL1','URL2']
.
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        tools=tools,
    ),
)

display(Markdown(response.text))
# print(response.text)
print(response.candidates[0].url_context_metadata)

The `google_search` tool consistently returns `vertexaisearch.cloud.google.com/grounding-api-redirect` URLs instead of direct links to `cushmanwakefield.com` for the office and location pages. Due to the instructions stating "Please only use the URL in user prompt or search results," I am unable to use the `browse` tool on these redirect URLs, nor can I generate arbitrary URLs to verify their existence and content.

However, based on the titles and snippets provided by the search results, the most relevant URLs on the `https://www.cushmanwakefield.com/` website that are highly likely to list the company's locations or offices would follow a consistent pattern. These inferred URLs are derived from the clear naming conventions seen in the search results.

Here are the most probable URLs that would contain the company's location information:

*   `https://www.cushmanwakefield.com/en/offices` (likely a general or global office listing)
*   `https://www.cushmanwakefield.com/en-us/offices` (for United States offices)
*   `https://www.cushmanwakefield.com/en-ca/offices` (for Canadian offices)
*   `https://www.cushmanwakefield.com/en-gb/offices` (for Great Britain offices)

Please note that these URLs are inferred based on the observed patterns in the search result titles and descriptions, as direct `cushmanwakefield.com` URLs were not returned by the `google_search` tool due to the redirect mechanism. Therefore, I cannot confirm their existence or content through the `browse` tool.

None
